<a href="https://colab.research.google.com/github/addaia/TechnicalProject/blob/main/ML_GSM8k.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**4 Supervised non LLMs**

In [2]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 11.4 MB/s eta 0:00:00


In [3]:
# imports
import os
import re
import itertools
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datasets import load_dataset
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from IPython.display import display

In [ ]:
# define models and parameters to 'fine tune'
models = {
    'LogisticRegression': {
        'C': [0.01, 0.1, 1, 10, 100],
        'max_iter': [500, 1000, 2000],
        'solver': ['lbfgs', 'liblinear']
    },
    'RandomForest': {
        'n_estimators': [50, 100, 200, 300],
        'max_depth': [5, 10, 20, None],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4]
    },
    'SVM': {
        'C': [0.01, 0.1, 1, 10, 100],
        'kernel': ['linear', 'rbf', 'poly'],
        'gamma': ['scale', 'auto']
    }
}

# preprocess dataset:
#   question -- same
#   answer ---- number (only)
def preprocess_gsm8k(dataset):
    preprocessed_data = []
    for example in dataset:
        question = example["question"]
        answer = example["answer"]
        extracted_answer = re.findall(r"####\s*(\d+)", answer)
        if extracted_answer:
            formatted_answer = extracted_answer[-1]
            preprocessed_data.append({"question": question, "answer": formatted_answer})
    return preprocessed_data

# preprocess dataset -- vectorise inputs to be able to enter algortihms
def load_and_vectorize_data():
    dataset = load_dataset("gsm8k", "main")
    train_data = preprocess_gsm8k(dataset['train'])
    test_data = preprocess_gsm8k(dataset.get('test', dataset.get('validation')))

    train_df = pd.DataFrame(train_data)
    test_df = pd.DataFrame(test_data)

    vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)
    X_train = vectorizer.fit_transform(train_df['question'])
    X_test = vectorizer.transform(test_df['question'])

    y_train = train_df['answer'].astype(int)
    y_test = test_df['answer'].astype(int)

    return X_train, X_test, y_train, y_test, vectorizer

# train and evaluate
def train_and_evaluate(X_train, X_test, y_train, y_test, model_name, params):
    if model_name == 'LogisticRegression':
        model = LogisticRegression(**params)
    elif model_name == 'RandomForest':
        model = RandomForestClassifier(**params)
    elif model_name == 'SVM':
        model = SVC(**params)

    # calculate time taken
    start_time = time.time()
    model.fit(X_train, y_train)
    elapsed_time = time.time() - start_time

    # record aaccuracy
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred, output_dict=True)

    return accuracy, report, elapsed_time

# hyperparameter Sweep
X_train, X_test, y_train, y_test, vectorizer = load_and_vectorize_data()

results = []
if X_train is not None:
    for model_name, param_grid in models.items():
        keys, values = zip(*param_grid.items())
        for combination in itertools.product(*values):
            params = dict(zip(keys, combination))
            accuracy, report, elapsed_time = train_and_evaluate(X_train, X_test, y_train, y_test, model_name, params)
            results.append({
                'Model': model_name,
                'Parameters': params,
                'Accuracy': accuracy,
                'TimeTaken': elapsed_time,
                'Precision': report['macro avg']['precision'],
                'Recall': report['macro avg']['recall'],
                'F1-score': report['macro avg']['f1-score']
            })

# to df
results_df = pd.DataFrame(results)

# disp best params per model
best_results = results_df.loc[results_df.groupby('Model')['Accuracy'].idxmax()]
display(best_results)

# save if wanted
# results_df.to_csv('hyperparameter_sweep_results.csv', index=False)

# plot
plt.figure(figsize=(10, 6))
colors = {'LogisticRegression': 'royalblue', 'RandomForest': 'forestgreen', 'SVM': 'tomato'}

for model_name in models.keys():
    subset = results_df[results_df['Model'] == model_name]
    plt.scatter(subset['TimeTaken'], subset['Accuracy'], label=model_name, color=colors[model_name], alpha=0.7)

plt.title('Time Taken vs Accuracy for Each Model')
plt.xlabel('Time Taken (seconds)')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)
plt.show()


Training on gsm8k dataset...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:86: UserWarning: 
Access to the secret `HF_TOKEN` has not been granted on this notebook.
You will not be requested again.
Please restart the session if you want to be prompted again.
  warnings.warn(


README.md:   0%|          | 0.00/7.94k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/2.31M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/419k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7473 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1319 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/m